In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import stats

from result_analysis.helper_functions import (
  process_csv_data,
  merge_data,
  classify_fuzzing_results,
)

In [ ]:
# Load experiment results
experiment1_results = process_csv_data("/home/csoare/experiment1/fuzz_results_with_cpog.csv")
experiment1_features = process_csv_data("/home/csoare/experiment1/features_output.csv")

white_cmap = sns.light_palette("white", as_cmap=True)


merged_data = merge_data(experiment1_results, experiment1_features)

print(merged_data.columns)
print(merged_data["counter"].unique())
print(merged_data["generator"].unique())
print(merged_data["cpog_message"].unique())

In [ ]:
# Separate into subsets
timeout_results, crash_results, correct_results, incorrect_results, *cpog_subsets = classify_fuzzing_results(merged_data)

merged_data["crashed"] = False
if 'crash_results' in locals():
    merged_data.loc[merged_data.index.isin(crash_results.index), "crashed"] = True

# Build (original_label, df) list
subset_label_pairs = [
    ("timeout_results", timeout_results),
    ("crash_results", crash_results),
    ("correct_results", correct_results),
    ("incorrect_results", incorrect_results),
]
for i, cpog_df in enumerate(cpog_subsets, start=1):
    if not cpog_df.empty:
        msg = cpog_df["cpog_message"].iloc[0]
    else:
        msg = f"CPOG_Message_{i}"
    subset_label_pairs.append((f"CPOG: {msg}", cpog_df))

# Map original labels to final labels
def map_label(original_label: str) -> str:
    if original_label == "timeout_results":
        return "timeout"
    elif original_label == "crash_results":
        return "crash"
    elif original_label == "correct_results":
        return "correct count"
    elif original_label == "incorrect_results":
        return "incorrect count"
    elif original_label.startswith("CPOG: "):
        # Remove "CPOG: " and any ":" character
        return original_label.replace("CPOG: ", "").replace(":", "")
    else:
        return original_label

# Gather all data into a list of (final_label, counter_val, count_val), grouping CPOG errors
all_data_grouped = []
for orig_label, df in subset_label_pairs:
    if "counter" not in df.columns:
        continue
    if orig_label.startswith("CPOG: "):
        final_label = "CPOG errors"  # Group all CPOG-related data under this label
    else:
        final_label = map_label(orig_label)
    # Count how many rows for each 'counter' value
    counts_series = df.groupby("counter").size() if not df.empty else pd.Series()
    for counter_val, count_val in counts_series.items():
        all_data_grouped.append((final_label, counter_val, count_val))

# Aggregate duplicate entries by summing counts
counts_grouped_df = (
    pd.DataFrame(all_data_grouped, columns=["subset_label", "counter_val", "count_val"])
    .groupby(["subset_label", "counter_val"], as_index=False)
    .sum()
)

pivot_grouped_df = (
    counts_grouped_df
    .pivot(index="counter_val", columns="subset_label", values="count_val")
    .fillna(0)
)

# Ensure all columns appear, even if empty
desired_col_order_grouped = [
    "correct count",
    "incorrect count",
    "crash",
    "timeout",
    "CPOG errors"
]
pivot_grouped_df = pivot_grouped_df.reindex(columns=desired_col_order_grouped, fill_value=0)


# Plot the grouped version
fig, ax = plt.subplots(figsize=(6, 6))

sns.heatmap(
    pivot_grouped_df.T,
    annot=True,
    fmt="g",
    cmap=white_cmap,
    cbar=False,
    linecolor="black",
    linewidths=1.0,
    annot_kws={"size": 18},
    ax=ax
)

ax.set_title("Distribution of Instance Types by Counter", fontsize=16, pad=20)
ax.xaxis.set_label_position("top")
ax.set_ylabel("Instance Type", fontsize=14)
ax.set_xlabel("Counter Name", fontsize=14)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()

# For detailed CPOG errors
cpog_data = []
for orig_label, df in subset_label_pairs:
    if orig_label.startswith("CPOG: ") and "counter" in df.columns:
        final_label = map_label(orig_label)
        counts_series = df.groupby("counter").size() if not df.empty else pd.Series()
        for counter_val, count_val in counts_series.items():
            cpog_data.append((final_label, counter_val, count_val))

if cpog_data:  # Only create second plot if there are CPOG errors
    cpog_df = (
        pd.DataFrame(cpog_data, columns=["subset_label", "counter_val", "count_val"])
        .groupby(["subset_label", "counter_val"], as_index=False)
        .sum()
    )

    pivot_cpog_df = (
        cpog_df
        .pivot(index="counter_val", columns="subset_label", values="count_val")
        .fillna(0)
    )

    fig, ax = plt.subplots(figsize=(6, 6))

    sns.heatmap(
        pivot_cpog_df.T,
        annot=True,
        fmt="g",
        cmap=white_cmap,
        cbar=False,
        linecolor="black",
        linewidths=1.0,
        annot_kws={"size": 18},
        ax=ax
    )
    ax.set_title("Distribution of CPOG Errors by Message and Counter", fontsize=16, pad=20)
    ax.xaxis.set_label_position("top")
    ax.set_xlabel("Counter Name", fontsize=14)
    ax.set_ylabel("CPOG Message", fontsize=14)
    plt.xticks(rotation=45, ha="right", fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.show()

In [ ]:
all_data_grouped = []
unique_counters = merged_data["counter"].nunique()
for orig_label, df in subset_label_pairs:
  if "generator" not in df.columns or "counter" not in df.columns:
    continue
  if orig_label.startswith("CPOG: "):
    final_label = "CPOG errors"
  else:
    final_label = map_label(orig_label)
  counts_series = df.groupby("generator").size().apply(lambda x: int(x / unique_counters))
  for generator_val, count_val in counts_series.items():
    all_data_grouped.append((final_label, generator_val, count_val))

counts_grouped_df = (
  pd.DataFrame(all_data_grouped, columns=["subset_label", "generator_val", "count_val"])
  .groupby(["subset_label", "generator_val"], as_index=False)
  .sum()
)

pivot_grouped_df = (
  counts_grouped_df
  .pivot(index="generator_val", columns="subset_label", values="count_val")
  .fillna(0)
)


ordered_generators = [
  'FuzzSAT-easy',
  'FuzzSAT-medium',
  'FuzzSAT-hard',
  'FuzzSAT-mixed',
  'FuzzSAT-random-medium',
  'FuzzSAT-random-hard',
  'FuzzSAT-structured-hard'
]
pivot_grouped_df = pivot_grouped_df.reindex(columns=desired_col_order_grouped, index=ordered_generators, fill_value=0)

fig, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(
  pivot_grouped_df.T,
  fmt="g",
  annot=True,
  cmap=white_cmap,
  cbar=False,
  linecolor="black",
  linewidths=1.0,
  annot_kws={"size": 20},
  ax=ax
)
ax.set_title("Distribution of Instance Types by Generator (Grouped CPOG Errors)", fontsize=14, pad=20)
ax.xaxis.set_label_position("top")
ax.yaxis.set_ticks_position("left")
ax.set_ylabel("Instance Type", fontsize=12)
ax.set_xlabel("Generator Name", fontsize=12)
plt.xticks(rotation=30, fontsize=10, ha='right')
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()

cpog_data = []
for orig_label, df in subset_label_pairs:
  if orig_label.startswith("CPOG: ") and "generator" in df.columns and "counter" in df.columns:
    final_label = map_label(orig_label)
    unique_counters = df["counter"].nunique()
    counts_series = df.groupby("generator").size().apply(lambda x: int(math.ceil(x / unique_counters)))
    for generator_val, count_val in counts_series.items():
      cpog_data.append((final_label, generator_val, count_val))

cpog_df = (
  pd.DataFrame(cpog_data, columns=["subset_label", "generator_val", "count_val"])
  .groupby(["subset_label", "generator_val"], as_index=False)
  .sum()
)

pivot_cpog_df = (
  cpog_df
  .pivot(index="generator_val", columns="subset_label", values="count_val")
  .fillna(0)
)

pivot_cpog_df = pivot_cpog_df.reindex(index=ordered_generators, fill_value=0)

fig, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(
  pivot_cpog_df.T,
  annot=True,
  cmap=white_cmap,
  cbar=False,
  linecolor="black",
  linewidths=1.0,
  annot_kws={"size": 20},
  ax=ax
)
ax.set_title("Distribution of CPOG Errors by Message and Generator", fontsize=14, pad=20)
ax.xaxis.set_label_position("top")
ax.yaxis.set_ticks_position("left")
ax.set_xlabel("Generator Name", fontsize=12)
ax.set_ylabel("CPOG Message", fontsize=12)
plt.xticks(rotation=30, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.show()

In [5]:
# Create a pivot table combining count_matches, timed_out, and cpog_message
behavior_matrix = merged_data.pivot_table(
   index=["generator", "instance_name"],
   columns="counter", 
   values=["count_matches", "timed_out", "cpog_message", "crashed"],
   aggfunc="first"
).fillna("missing")

# Initialize list to store similarity data
similarity_data = []

# Calculate similarities between generators with different metrics
for i, gen1 in enumerate(ordered_generators):
   for j, gen2 in enumerate(ordered_generators):
       if i >= j: # Only compute upper triangle
           continue
           
       gen1_behavior = behavior_matrix[behavior_matrix.index.get_level_values("generator") == gen1]
       gen2_behavior = behavior_matrix[behavior_matrix.index.get_level_values("generator") == gen2]
       instance_similarities = []
       
       for idx1, row1 in gen1_behavior.iterrows():
            for idx2, row2 in gen2_behavior.iterrows():
                matches = (row1 == row2).sum()
                comparisons = len(row1)
                instance_similarity = matches / comparisons if comparisons > 0 else 0
                instance_similarities.append(instance_similarity)
               
       # Calculate different aggregations
       similarity_data.append({
           'generator1': gen1,
           'generator2': gen2,
           'mean_similarity': np.mean(instance_similarities),
           'median_similarity': np.median(instance_similarities),
           'min_similarity': np.min(instance_similarities),
           'max_similarity': np.max(instance_similarities),
           'std_similarity': np.std(instance_similarities),
           'q25_similarity': np.percentile(instance_similarities, 25),
           'q75_similarity': np.percentile(instance_similarities, 75),
       })

# Convert to DataFrame
similarity_features_df = pd.DataFrame(similarity_data)

# Create symmetric version by adding reversed pairs
reversed_pairs = similarity_features_df.copy()
reversed_pairs[['generator1', 'generator2']] = reversed_pairs[['generator2', 'generator1']]
similarity_features_df = pd.concat([similarity_features_df, reversed_pairs])

In [ ]:
# Create heatmaps for each metric
metrics = ['mean_similarity', 'median_similarity']
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

fig.suptitle("Generator Similarity Analysis", fontsize=14, y=1.05)

for idx, metric in enumerate(metrics):
    # Create similarity matrix for this metric
    pivot_df = similarity_features_df.pivot(index='generator1',
                                          columns='generator2',
                                          values=metric)
    
    # Reindex with ordered generators
    pivot_df = pivot_df.reindex(index=ordered_generators,
                               columns=ordered_generators)
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(pivot_df, dtype=bool))
    
    # Create heatmap - only show cbar for the last plot
    sns.heatmap(pivot_df,
                xticklabels=ordered_generators,
                yticklabels=ordered_generators if idx == 0 else [],
                cmap=plt.cm.viridis,
                mask=mask,
                annot=True,
                fmt=".2f",
                square=True,
                ax=axes[idx],
                cbar=idx == 1)  # Changed from idx == 2 to idx == 1 since we only have 2 plots
    
    # Force aspect ratio to be equal
    axes[idx].set_aspect('equal')
    axes[idx].set_title(metric)
    axes[idx].set_xticklabels(ordered_generators, rotation=45, ha="right")

# Adjust layout while maintaining aspect ratio
plt.tight_layout()
plt.show()

In [ ]:
# Select numeric features
exclude_cols = ["counter", "satisfiability", "problem_type", "timed_out",
                "error", "generator", "verified", "cpog_message", "cpog_count",
                "count_matches", "solved", "seed", "result_type"]
feature_cols = [col for col in merged_data.columns 
                if col not in exclude_cols and 
                merged_data[col].dtype in ["float64", "int64"]]

# Get similarity metrics and their specific analysis methods
similarity_metrics = {
    'mean_similarity': 'median',  # use median for robustness
    'median_similarity': 'median',  # natural split point
    'min_similarity': 'quartile',  # compare bottom 25% vs top 25%
    'max_similarity': 'quartile',  # compare bottom 25% vs top 25%
    'std_similarity': 'quartile',  # compare high/low variance groups
    'q25_similarity': 'fixed',  # use q25 value as fixed threshold
    'q75_similarity': 'fixed'   # use q75 value as fixed threshold
}

all_metric_results = {}

for similarity_metric, threshold_type in similarity_metrics.items():
    feature_stats = []
    max_sample = 1000
    
    for feature in feature_cols:
        feature_diffs = []
        similarities = []
        
        # Compute feature differences between generators
        for i, gen1 in enumerate(ordered_generators):
            for j, gen2 in enumerate(ordered_generators[i+1:], i+1):
                sim_score = similarity_features_df[
                    (similarity_features_df['generator1'] == gen1) & 
                    (similarity_features_df['generator2'] == gen2)
                ][similarity_metric].iloc[0]
                
                gen1_data = merged_data[merged_data["generator"] == gen1][feature]
                gen2_data = merged_data[merged_data["generator"] == gen2][feature]
                
                if len(gen1_data) > max_sample:
                    gen1_data = gen1_data.sample(max_sample)
                if len(gen2_data) > max_sample:
                    gen2_data = gen2_data.sample(max_sample)
                
                diffs = np.abs(gen1_data.values[:, np.newaxis] - gen2_data.values[np.newaxis, :]).flatten()
                
                if len(diffs) > max_sample:
                    diffs = np.random.choice(diffs, max_sample, replace=False)
                
                feature_diffs.extend(diffs)
                similarities.extend([sim_score] * len(diffs))
        
        # Skip if no data collected
        if not feature_diffs or not similarities:
            continue
            
        feature_diffs = np.array(feature_diffs)
        similarities = np.array(similarities)
        
        # Apply different thresholding strategies
        if threshold_type == 'median':
            threshold = np.median(similarities)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
            
        elif threshold_type == 'quartile':
            q75, q25 = np.percentile(similarities, [75, 25])
            high_sim_mask = similarities >= q75
            low_sim_mask = similarities <= q25
            
        elif threshold_type == 'fixed':
            if similarity_metric == 'q25_similarity':
                threshold = np.percentile(similarities, 25)
            else:  # q75_similarity
                threshold = np.percentile(similarities, 75)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
        
        high_sim = feature_diffs[high_sim_mask]
        low_sim = feature_diffs[low_sim_mask]
        
        min_samples = 30
        if len(high_sim) < min_samples or len(low_sim) < min_samples:
            continue
            
        sample_size = min(len(high_sim), len(low_sim), max_sample)
        high_sim = np.random.choice(high_sim, sample_size, replace=False)
        low_sim = np.random.choice(low_sim, sample_size, replace=False)
        
        if len(np.unique(high_sim)) == 1 or len(np.unique(low_sim)) == 1:
            continue
            
        statistic, p_value = stats.mannwhitneyu(high_sim, low_sim, alternative='two-sided')
        cliff_delta = abs((2 * statistic - sample_size * sample_size) / (sample_size * sample_size))
        
        feature_stats.append({
            "feature": feature,
            "p_value": p_value,
            "effect_size": cliff_delta,
            "test_type": "Mann-Whitney",
            "threshold_type": threshold_type
        })
    
    # Skip if no statistics were collected
    if not feature_stats:
        print(f"\nNo significant features found for {similarity_metric}")
        continue
        
    alpha = 0.05
    feature_stats_df = pd.DataFrame(feature_stats)
    feature_stats_df["significant"] = feature_stats_df["p_value"] < (alpha / len(feature_cols))
    feature_stats_df = feature_stats_df.sort_values("effect_size", ascending=False)
    
    significant_features = feature_stats_df[feature_stats_df["significant"]]
    if not significant_features.empty:
        all_metric_results[similarity_metric] = significant_features
    else:
        print(f"\nNo significant features found for {similarity_metric} after correction")

# Print top significant features for each similarity metric
for metric, results_df in all_metric_results.items():
    print(f"\nTop significant features for {metric} (using {similarity_metrics[metric]} thresholding):")
    print(results_df.head(5)[["feature", "effect_size", "test_type", "threshold_type"]])

In [ ]:
# Create visualizations for each similarity metric
for metric, results_df in all_metric_results.items():
    top_features = results_df[results_df["significant"]].head(5)["feature"].tolist()
    if not top_features:
        continue
        
    n_features = len(top_features)
    fig, axes = plt.subplots(n_features, 2, figsize=(15, 5*n_features))
    
    # Ensure axes is always a 2D array
    if n_features == 1:
        axes = axes.reshape(1, -1)
    
    # Create proper viridis colors
    viridis = plt.cm.viridis
    colors = [viridis(0.2), viridis(0.8)]
    
    for idx, feature in enumerate(top_features):
        # Recompute feature differences and similarities
        feature_diffs = []
        similarities = []
        
        for i, gen1 in enumerate(ordered_generators):
            for j, gen2 in enumerate(ordered_generators[i+1:], i+1):
                # Get similarity score for this generator pair
                sim_score = similarity_features_df[
                    (similarity_features_df["generator1"] == gen1) & 
                    (similarity_features_df["generator2"] == gen2)
                ][metric].iloc[0]
                
                # Get feature data for both generators
                gen1_data = merged_data[merged_data["generator"] == gen1][feature]
                gen2_data = merged_data[merged_data["generator"] == gen2][feature]
                
                # Compute pairwise absolute differences
                diffs = np.abs(gen1_data.values[:, np.newaxis] - gen2_data.values[np.newaxis, :]).flatten()
                
                # Store differences and corresponding similarity
                feature_diffs.extend(diffs)
                similarities.extend([sim_score] * len(diffs))
        
        # Convert to numpy arrays
        feature_diffs = np.array(feature_diffs)
        similarities = np.array(similarities)
        
        # Apply the same thresholding strategy as in the analysis
        threshold_type = similarity_metrics[metric]
        if threshold_type == "median":
            threshold = np.median(similarities)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
        elif threshold_type == "quartile":
            q75, q25 = np.percentile(similarities, [75, 25])
            high_sim_mask = similarities >= q75
            low_sim_mask = similarities <= q25
        elif threshold_type == "fixed":
            if metric == "q25_similarity":
                threshold = np.percentile(similarities, 25)
            else:  # q75_similarity
                threshold = np.percentile(similarities, 75)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
            
        high_sim_data = feature_diffs[high_sim_mask]
        low_sim_data = feature_diffs[low_sim_mask]
        
        # Boxplot
        ax_box = axes[idx, 0]
        bp = ax_box.boxplot([low_sim_data, high_sim_data],
                          tick_labels=["Low Similarity", "High Similarity"],
                          patch_artist=True)
        
        # Set proper viridis colors
        for patch, color in zip(bp["boxes"], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.6)
            
        plt.setp(bp["whiskers"], color=colors[0])
        plt.setp(bp["caps"], color=colors[0])
        plt.setp(bp["medians"], color="white")
        plt.setp(bp["fliers"], markerfacecolor=colors[0])
        
        ax_box.set_title(f"{feature} - Boxplot")
        
        # Density plot
        ax_density = axes[idx, 1]
        x_range = np.linspace(min(feature_diffs), max(feature_diffs), 200)
        
        if len(np.unique(low_sim_data)) > 1:
            kde_low = stats.gaussian_kde(low_sim_data)
            ax_density.fill_between(x_range, kde_low(x_range),
                                  alpha=0.6, color=colors[0],
                                  label="Low Similarity")
            
        if len(np.unique(high_sim_data)) > 1:
            kde_high = stats.gaussian_kde(high_sim_data)
            ax_density.fill_between(x_range, kde_high(x_range),
                                  alpha=0.6, color=colors[1],
                                  label="High Similarity")
            
        ax_density.set_title(f"{feature} - Density Plot")
        ax_density.legend()
    
    fig.suptitle(f"Feature Distributions for {metric}\n(using {threshold_type} thresholding)", 
                 fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Print statistical results
    print(f"\nTop features by effect size for {metric} (statistically significant):")
    print(results_df[["feature", "test_type", "effect_size", "p_value", "threshold_type"]].head().to_string())

In [ ]:
# Set global font sizes for all plots
plt.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'legend.fontsize': 11,
    'figure.titlesize': 16
})

# Create visualizations for each similarity metric
for metric, results_df in all_metric_results.items():
    # Get only the first 2 significant features
    top_features = results_df[results_df["significant"]].head(2)["feature"].tolist()
    if not top_features:
        continue
        
    n_features = len(top_features)
    # Increased figure size and adjusted height per feature
    fig, axes = plt.subplots(n_features, 2, figsize=(16, 6*n_features))
    
    # Ensure axes is always a 2D array
    if n_features == 1:
        axes = axes.reshape(1, -1)
    
    # Create proper viridis colors with increased contrast
    viridis = plt.cm.viridis
    colors = [viridis(0.15), viridis(0.85)]
    
    for idx, feature in enumerate(top_features):
        # [Previous feature difference and similarity computation code remains the same]
        feature_diffs = []
        similarities = []
        
        for i, gen1 in enumerate(ordered_generators):
            for j, gen2 in enumerate(ordered_generators[i+1:], i+1):
                sim_score = similarity_features_df[
                    (similarity_features_df["generator1"] == gen1) & 
                    (similarity_features_df["generator2"] == gen2)
                ][metric].iloc[0]
                
                gen1_data = merged_data[merged_data["generator"] == gen1][feature]
                gen2_data = merged_data[merged_data["generator"] == gen2][feature]
                
                diffs = np.abs(gen1_data.values[:, np.newaxis] - gen2_data.values[np.newaxis, :]).flatten()
                
                feature_diffs.extend(diffs)
                similarities.extend([sim_score] * len(diffs))
        
        feature_diffs = np.array(feature_diffs)
        similarities = np.array(similarities)
        
        # Apply thresholding
        threshold_type = similarity_metrics[metric]
        if threshold_type == "median":
            threshold = np.median(similarities)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
        elif threshold_type == "quartile":
            q75, q25 = np.percentile(similarities, [75, 25])
            high_sim_mask = similarities >= q75
            low_sim_mask = similarities <= q25
        elif threshold_type == "fixed":
            if metric == "q25_similarity":
                threshold = np.percentile(similarities, 25)
            else:  # q75_similarity
                threshold = np.percentile(similarities, 75)
            high_sim_mask = similarities > threshold
            low_sim_mask = similarities <= threshold
            
        high_sim_data = feature_diffs[high_sim_mask]
        low_sim_data = feature_diffs[low_sim_mask]
        
        # Enhanced Boxplot
        ax_box = axes[idx, 0]
        bp = ax_box.boxplot([low_sim_data, high_sim_data],
                          tick_labels=["Low Similarity", "High Similarity"],
                          patch_artist=True,
                          medianprops=dict(color="white", linewidth=1.5),
                          flierprops=dict(marker='o', markerfacecolor=colors[0], markersize=8),
                          widths=0.7)
        
        # Set proper viridis colors with higher opacity
        for patch, color in zip(bp["boxes"], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
            
        plt.setp(bp["whiskers"], color="black", linewidth=1.5)
        plt.setp(bp["caps"], color="black", linewidth=1.5)
        
        ax_box.set_title(f"{feature} - Boxplot", pad=15)
        ax_box.grid(True, linestyle='--', alpha=0.3)
        
        # Enhanced Density plot
        ax_density = axes[idx, 1]
        x_range = np.linspace(min(feature_diffs), max(feature_diffs), 200)
        
        if len(np.unique(low_sim_data)) > 1:
            kde_low = stats.gaussian_kde(low_sim_data)
            ax_density.fill_between(x_range, kde_low(x_range),
                                  alpha=0.7, color=colors[0],
                                  label="Low Similarity")
            
        if len(np.unique(high_sim_data)) > 1:
            kde_high = stats.gaussian_kde(high_sim_data)
            ax_density.fill_between(x_range, kde_high(x_range),
                                  alpha=0.7, color=colors[1],
                                  label="High Similarity")
            
        ax_density.set_title(f"{feature} - Density Plot", pad=15)
        ax_density.legend(frameon=True, edgecolor='black')
        ax_density.grid(True, linestyle='--', alpha=0.3)
    
    # Enhanced title and layout
    fig.suptitle(f"Feature Distributions for {metric}\n(using {threshold_type} thresholding)", 
                 fontsize=16, y=1.04)
    
    # Add more spacing between subplots
    plt.tight_layout()
    plt.subplots_adjust(top=0.88, hspace=0.3)
    plt.show()
    
    # Print statistical results for only top 2 features
    print(f"\nTop 2 features by effect size for {metric} (statistically significant):")
    print(results_df[["feature", "test_type", "effect_size", "p_value", "threshold_type"]].head(2).to_string())